In [2]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import io
from PIL import Image

In [3]:
df = pd.read_csv('dados/public.csv')
df_ap = pd.read_csv('dados\AerodromosPublicos.csv', sep=';')
df_ap = df_ap[['Código OACI', 'LATGEOPOINT','LONGEOPOINT']]
df = df.merge(df_ap, left_on='origem', right_on='Código OACI',how='left')

<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Eduardo\AppData\Local\Temp\ipykernel_5828\3691857365.py:2: SyntaxWarning: invalid escape sequence '\A'
  df_ap = pd.read_csv('dados\AerodromosPublicos.csv', sep=';')


In [4]:
df_train = df[~df.espera.isnull()]
df_test = df[df.espera.isnull()]

In [5]:
def load_crop_img(path):
    image = Image.open(path)
    w = image.width
    h = image.height
    img_brasil = image.crop((1030, 820, w-250, h-620))
    return img_brasil

In [6]:
def paint_airport(image, lat_airport, lon_airport):

    # Mapas de latitude e longitude do Brasil
    lat_map = np.linspace(-33.750812, 5.235216, 888)
    lon_map = np.linspace(-73.981645, -34.792202, 912)

    # Posicao do aeroporto na imagem
    lat_result = np.abs(lat_map - lat_airport).argmin()
    lon_result = np.abs(lon_map - lon_airport).argmin()

    # Ajuste da latitude
    lat_result = 888 - lat_result

    # Pinta a imagem
    image[lat_result - 10 : lat_result + 10, lon_result - 10 : lon_result + 10 , :] = [148,0,211] # Roxo

    return image

In [8]:
for id, url, espera, lat, lon in tqdm(zip(df_train.flightid ,df_train.url_img_satelite, df_train.espera, df_train.LATGEOPOINT, df_train.LONGEOPOINT)):
    try:
        image_name = url.split('/')[-1]
        image = load_crop_img('imgs/' + image_name)
        np_img = np.array(image)
        img_ap = paint_airport(np_img, float(lat), float(lon))
        image_final = Image.fromarray(img_ap)
        image_final = image_final.resize((224,224))
    except:
        continue
    if espera:
        label = 'atrasado'
    else:
        label = 'pontual'
    image_final.save(f'dados/imgs/{label}/{id}.jpg')

211679it [1:32:15, 38.24it/s]  


In [11]:
for id, url, lat, lon in tqdm(zip(df_test.flightid ,df_test.url_img_satelite, df_test.LATGEOPOINT, df_test.LONGEOPOINT)):
    try:
        image_name = url.split('/')[-1]
        image = load_crop_img('imgs/' + image_name)
        np_img = np.array(image)
        img_ap = paint_airport(np_img, float(lat), float(lon))
        image_final = Image.fromarray(img_ap)
        image_final = image_final.resize((224,224))
    except:
        continue
    image_final.save(f'dados/test/{id}.jpg')

90720it [42:58, 35.19it/s] 
